# Span Batch Analysis - Data Preparation

## Initialization

Before start, Fill in [secret.yml](span-batch-analysis/secret.yaml). Refer to [secret_example.yml](span-batch-analysis/secret_example.yaml). This is for specifying RPC endpoints.

Also check config located at [config](span-batch-analysis/config/).

In [1]:
import os

Build binaries.

In [ ]:
%%bash
cd cmd/batch_decoder && go build . && cd ../../
cd cmd/span_batch_tester && go build . && cd ../../

In [2]:
%%capture
%env BATCH_DECODER=cmd/batch_decoder/batch_decoder
%env SPAN_BATCH_TESTER=cmd/span_batch_tester/span_batch_tester

Clone superchain config.

In [ ]:
%%bash
git clone https://github.com/ethereum-optimism/superchain-registry
cd superchain-registry && git checkout 5d3ca1966435827ac8c4216bddcb541ec64c1d6c && cd --

Select your chain.

In [3]:
%%capture
%env L1_CHAIN_NAME=mainnet
%env L2_CHAIN_NAME=zora

# available networks
#      L1 - L2 list
# mainnet - base, op, pgn, zora
# goerli  - base, op, zora
# sepolia - op, pgn

In [4]:
l1_chain_name = os.environ["L1_CHAIN_NAME"]
l2_chain_name = os.environ["L2_CHAIN_NAME"]

start_number = !yq eval ".start_l1_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
start_number_value = int(start_number[0])
end_number = !yq eval ".end_l1_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
end_number_value = int(end_number[0])
batch_tx_sender = !yq eval ".batch_tx_sender" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
batch_tx_sender_value = batch_tx_sender[0]
l2_genesis_timestamp = !yq eval ".genesis.l2_time" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
l2_genesis_timestamp_value = int(l2_genesis_timestamp[0])
l2_chain_id = !yq eval ".chain_id" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
l2_chain_id_value = int(l2_chain_id[0])
inbox_addr = !yq eval ".batch_inbox_addr" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
inbox_addr_value = inbox_addr[0]

acc_start_number = !yq eval ".acc_l2_start_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
acc_start_number_value = int(acc_start_number[0])
acc_end_number = !yq eval ".acc_l2_end_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
acc_end_number_value = int(acc_end_number[0])

Read analyis config.

In [5]:
%env L1_START=$start_number_value
%env L1_END=$end_number_value
%env BATCH_TX_SENDER=$batch_tx_sender_value
%env L2_GENESIS_TIMESTAMP=$l2_genesis_timestamp_value
%env L2_CHAIN_ID=$l2_chain_id_value
%env INBOX=$inbox_addr_value

%env ACC_L2_START=$acc_start_number_value
%env ACC_L2_END=$acc_end_number_value


env: L1_START=18000000
env: L1_END=18020000
env: BATCH_TX_SENDER=0x625726c858dBF78c0125436C943Bf4b4bE9d9033
env: L2_GENESIS_TIMESTAMP=1686693839
env: L2_CHAIN_ID=7777777
env: INBOX=0x6F54Ca6F6EdE96662024Ffd61BFd18f3f4e34DFf
env: ACC_L2_START=3348000
env: ACC_L2_END=3349000


Get L1 RPC from secret.

In [8]:
key_name = f"L1_{l1_chain_name}_RPC".upper()
L1_RPC = !yq eval ".{key_name}" span-batch-analysis/secret.yaml
L1_RPC_ENDPOINT = L1_RPC[0]

key_name = f"L2_{l2_chain_name}_{l1_chain_name}_RPC".upper()
L2_RPC = !yq eval ".{key_name}" span-batch-analysis/secret.yaml
L2_RPC_ENDPOINT = L2_RPC[0]

In [10]:
%%capture
%env L1_RPC=$L1_RPC_ENDPOINT
%env L2_RPC=$L2_RPC_ENDPOINT

## Data Collection

All data will be stored at `./artifacts/{L1_CHAIN_NAME}/{L2_CHAIN_NAME}`

In [11]:
l1_chain_name = os.environ["L1_CHAIN_NAME"]
l2_chain_name = os.environ["L2_CHAIN_NAME"]

%env DB_PREFIX=./artifacts/$l1_chain_name/$l2_chain_name

env: DB_PREFIX=./artifacts/mainnet/zora


batch decoder `fetch`: Read channel frames from L1 by reading batch transactions.

In [12]:
!$BATCH_DECODER fetch --l1=$L1_RPC --start=$L1_START --end=$L1_END --inbox=$INBOX --out=$DB_PREFIX/transactions_cache --sender=$BATCH_TX_SENDER --concurrent-requests=25 | tail -10

Fetched block:  18019998
Fetched block:  18019879
Fetched block:  18019993
Fetched block:  18019956
Fetched block:  18019889
Fetched block:  18019991
Fetched block:  18019977
Fetched batches in range [18000000,18020000). Found 408 valid & 0 invalid batches
Fetch Config: Chain ID: 1. Inbox Address: 0x6F54Ca6F6EdE96662024Ffd61BFd18f3f4e34DFf. Valid Senders: map[0x625726c858dBF78c0125436C943Bf4b4bE9d9033:{}].
Wrote transactions with batches to ./artifacts/mainnet/zora/transactions_cache


batch decoder `reassemble`: Reassemble channel frames to channels. 

In [13]:
!$BATCH_DECODER reassemble --inbox=$INBOX --in=$DB_PREFIX/transactions_cache --out=$DB_PREFIX/channel_cache | tail -10

span batch tester `convert`: Convert channels with singular batches to span batches. Try multiple tx encoding types.

In [14]:
!TX_ENCODE_TYPE=0  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f83fc6b32c38bcbdd3ae9c6f063b8801, L2StartNum: 3294052, L2EndNum: 3294351, L2BlockCnt, 300
[400/408] Channel ID: f869d64673aa2f4e04aef4edae39656c, L2StartNum: 3215067, L2EndNum: 3215361, L2BlockCnt, 295
[401/408] Channel ID: f8e958456d48e3f6a7419c9586ecdf23, L2StartNum: 3247154, L2EndNum: 3247453, L2BlockCnt, 300
[402/408] Channel ID: f9e7149e3f12f9b22f13d5be43afaecc, L2StartNum: 3245084, L2EndNum: 3245378, L2BlockCnt, 295
[403/408] Channel ID: fa9a9b2c115638768e19c086c1f19835, L2StartNum: 3187852, L2EndNum: 3188143, L2BlockCnt, 292
[404/408] Channel ID: fb4bf2a168cb421815dbe98e5f315acf, L2StartNum: 3235844, L2EndNum: 3236138, L2BlockCnt, 295
[405/408] Channel ID: fdd5ea1e64138e9746c6ee85def32272, L2StartNum: 3228122, L2EndNum: 3228416, L2BlockCnt, 295
[406/408] Channel ID: fe8aebaa27f7b356db104b6d8fcf710e, L2StartNum: 3229917, L2EndNum: 3230208, L2BlockCnt, 292
[407/408] Channel ID: fee1bb6ecbcc15d19026d6c0c1d02a8f, L2StartNum: 3292539, L2EndNum: 3292838, L2BlockC

In [15]:
!TX_ENCODE_TYPE=1  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f83fc6b32c38bcbdd3ae9c6f063b8801, L2StartNum: 3294052, L2EndNum: 3294351, L2BlockCnt, 300
[400/408] Channel ID: f869d64673aa2f4e04aef4edae39656c, L2StartNum: 3215067, L2EndNum: 3215361, L2BlockCnt, 295
[401/408] Channel ID: f8e958456d48e3f6a7419c9586ecdf23, L2StartNum: 3247154, L2EndNum: 3247453, L2BlockCnt, 300
[402/408] Channel ID: f9e7149e3f12f9b22f13d5be43afaecc, L2StartNum: 3245084, L2EndNum: 3245378, L2BlockCnt, 295
[403/408] Channel ID: fa9a9b2c115638768e19c086c1f19835, L2StartNum: 3187852, L2EndNum: 3188143, L2BlockCnt, 292
[404/408] Channel ID: fb4bf2a168cb421815dbe98e5f315acf, L2StartNum: 3235844, L2EndNum: 3236138, L2BlockCnt, 295
[405/408] Channel ID: fdd5ea1e64138e9746c6ee85def32272, L2StartNum: 3228122, L2EndNum: 3228416, L2BlockCnt, 295
[406/408] Channel ID: fe8aebaa27f7b356db104b6d8fcf710e, L2StartNum: 3229917, L2EndNum: 3230208, L2BlockCnt, 292
[407/408] Channel ID: fee1bb6ecbcc15d19026d6c0c1d02a8f, L2StartNum: 3292539, L2EndNum: 3292838, L2BlockC

In [16]:
!TX_ENCODE_TYPE=2  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f83fc6b32c38bcbdd3ae9c6f063b8801, L2StartNum: 3294052, L2EndNum: 3294351, L2BlockCnt, 300
[400/408] Channel ID: f869d64673aa2f4e04aef4edae39656c, L2StartNum: 3215067, L2EndNum: 3215361, L2BlockCnt, 295
[401/408] Channel ID: f8e958456d48e3f6a7419c9586ecdf23, L2StartNum: 3247154, L2EndNum: 3247453, L2BlockCnt, 300
[402/408] Channel ID: f9e7149e3f12f9b22f13d5be43afaecc, L2StartNum: 3245084, L2EndNum: 3245378, L2BlockCnt, 295
[403/408] Channel ID: fa9a9b2c115638768e19c086c1f19835, L2StartNum: 3187852, L2EndNum: 3188143, L2BlockCnt, 292
[404/408] Channel ID: fb4bf2a168cb421815dbe98e5f315acf, L2StartNum: 3235844, L2EndNum: 3236138, L2BlockCnt, 295
[405/408] Channel ID: fdd5ea1e64138e9746c6ee85def32272, L2StartNum: 3228122, L2EndNum: 3228416, L2BlockCnt, 295
[406/408] Channel ID: fe8aebaa27f7b356db104b6d8fcf710e, L2StartNum: 3229917, L2EndNum: 3230208, L2BlockCnt, 292
[407/408] Channel ID: fee1bb6ecbcc15d19026d6c0c1d02a8f, L2StartNum: 3292539, L2EndNum: 3292838, L2BlockC

span batch tester `analyze`: Analyze channels with singluar batches by comparing with corresponding span batches.

In [17]:
!TX_ENCODE_TYPE=0 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f83fc6b32c38bcbdd3ae9c6f063b8801, CompressedReductionPercent: 25.258686 %
[400/408] Channel ID: f869d64673aa2f4e04aef4edae39656c, CompressedReductionPercent: 25.056463 %
[401/408] Channel ID: f8e958456d48e3f6a7419c9586ecdf23, CompressedReductionPercent: 31.772283 %
[402/408] Channel ID: f9e7149e3f12f9b22f13d5be43afaecc, CompressedReductionPercent: 36.931331 %
[403/408] Channel ID: fa9a9b2c115638768e19c086c1f19835, CompressedReductionPercent: 24.811493 %
[404/408] Channel ID: fb4bf2a168cb421815dbe98e5f315acf, CompressedReductionPercent: 25.873518 %
[405/408] Channel ID: fdd5ea1e64138e9746c6ee85def32272, CompressedReductionPercent: 25.262607 %
[406/408] Channel ID: fe8aebaa27f7b356db104b6d8fcf710e, CompressedReductionPercent: 24.228526 %
[407/408] Channel ID: fee1bb6ecbcc15d19026d6c0c1d02a8f, CompressedReductionPercent: 22.106232 %
[408/408] Channel ID: ff16122882d3cc26a3111f32353f620f, CompressedReductionPercent: 28.473456 %


In [18]:
!TX_ENCODE_TYPE=1 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f83fc6b32c38bcbdd3ae9c6f063b8801, CompressedReductionPercent: 22.636929 %
[400/408] Channel ID: f869d64673aa2f4e04aef4edae39656c, CompressedReductionPercent: 21.925134 %
[401/408] Channel ID: f8e958456d48e3f6a7419c9586ecdf23, CompressedReductionPercent: 29.526119 %
[402/408] Channel ID: f9e7149e3f12f9b22f13d5be43afaecc, CompressedReductionPercent: 34.733825 %
[403/408] Channel ID: fa9a9b2c115638768e19c086c1f19835, CompressedReductionPercent: 22.145029 %
[404/408] Channel ID: fb4bf2a168cb421815dbe98e5f315acf, CompressedReductionPercent: 22.568315 %
[405/408] Channel ID: fdd5ea1e64138e9746c6ee85def32272, CompressedReductionPercent: 22.191346 %
[406/408] Channel ID: fe8aebaa27f7b356db104b6d8fcf710e, CompressedReductionPercent: 20.785123 %
[407/408] Channel ID: fee1bb6ecbcc15d19026d6c0c1d02a8f, CompressedReductionPercent: 19.961872 %
[408/408] Channel ID: ff16122882d3cc26a3111f32353f620f, CompressedReductionPercent: 25.239594 %


In [19]:
!TX_ENCODE_TYPE=2 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f83fc6b32c38bcbdd3ae9c6f063b8801, CompressedReductionPercent: 26.859354 %
[400/408] Channel ID: f869d64673aa2f4e04aef4edae39656c, CompressedReductionPercent: 26.756910 %
[401/408] Channel ID: f8e958456d48e3f6a7419c9586ecdf23, CompressedReductionPercent: 32.308977 %
[402/408] Channel ID: f9e7149e3f12f9b22f13d5be43afaecc, CompressedReductionPercent: 38.055807 %
[403/408] Channel ID: fa9a9b2c115638768e19c086c1f19835, CompressedReductionPercent: 27.058376 %
[404/408] Channel ID: fb4bf2a168cb421815dbe98e5f315acf, CompressedReductionPercent: 27.786189 %
[405/408] Channel ID: fdd5ea1e64138e9746c6ee85def32272, CompressedReductionPercent: 27.283441 %
[406/408] Channel ID: fe8aebaa27f7b356db104b6d8fcf710e, CompressedReductionPercent: 25.945377 %
[407/408] Channel ID: fee1bb6ecbcc15d19026d6c0c1d02a8f, CompressedReductionPercent: 23.931107 %
[408/408] Channel ID: ff16122882d3cc26a3111f32353f620f, CompressedReductionPercent: 30.323386 %


span batch tester `fetch`: Fetch consecutive singular batches.

In [20]:
!$SPAN_BATCH_TESTER fetch --l2 $L2_RPC --start=$ACC_L2_START --end=$ACC_L2_END --concurrent-requests=100 --out=$DB_PREFIX/singular_batches_cache | tail -10

Fetched L2 block:  3348988
Fetched L2 block:  3348987
Fetched L2 block:  3348993
Fetched L2 block:  3348996
Fetched L2 block:  3348995
Fetched L2 block:  3348998
Fetched L2 block:  3348997
Fetched L2 block:  3348992
Fetched v0 batches in range [3348000,3349000).
Fetch Config: Chain ID: 7777777.


span batch tester `merge`: Merge singular batches:

This command will analyze `start - end` number of batches, Comparing uncompressed size of below two batches:
- For `i` in range  `0` to `start - end - 1`
  - singular batch from number `start` to `start + i`, inclusive.
  - span batch from number `start` to `start + i`, inclusive.

Upper command will make a result json file which has name of the form `{start_number}_{end_number}.json`.

In [22]:
!$SPAN_BATCH_TESTER merge --start=$ACC_L2_START --end=$ACC_L2_END --l2 $L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --in=$DB_PREFIX/singular_batches_cache --out=$DB_PREFIX/merge_result | tail -10

Merged v0 batches in range [3348000,3349000).
